In [1]:
import pandas as pd
import re
import os

In [2]:
csv_path = r"C:\Users\Acer\Desktop\vs_paths.csv"
data_dir = r'D:\VSdata'
df = pd.read_csv(csv_path)

In [3]:
weight_folders = list(df['image_path'])
weight_folders = [os.path.join(data_dir,p) for p in weight_folders]
weight_folders = [p.replace("\\", "/") for p in weight_folders]
len(weight_folders)

301

In [4]:
weight_folders[6]

'D:/VSdata/Vestibular-Schwannoma-MC-RC/VS-SEG-013/12-23-1991-NA-t1of10External Images for PACS-95314/2.000000-T2T2 AXIAL CISS-10876'

In [5]:
seg_paths = list(df['SegmentationPath'])
seg_paths = [os.path.join(data_dir,p) for p in seg_paths]
seg_paths = [p.replace("\\", "/") for p in seg_paths]
len(seg_paths)

301

In [6]:
def extract_patient_name(path):
    pattern = r"/(?P<patient_name>(?:VS-(?:SEG|MC-RC)-)?\d+)/(?P<date_of_mri>\d{2}-\d{2}-\d{4})"
    match = re.search(pattern, path)
    if match:
        return match.group(1)
    return None

def extract_patient_year(path):
  pattern = r"/(?P<patient_name>(?:VS-(?:SEG|MC-RC)-)?\d+)/(?P<date_of_mri>\d{2}-\d{2}-\d{4})"
  match = re.search(pattern, path)
  if match:
    return match.group(2)
  return None

In [7]:
x = []

for i in range(len(weight_folders)):
# for i in range(6):

    patient_id = extract_patient_name(weight_folders[i])
    if patient_id is None:
        print(f"Warning: Could not extract patient ID from {weight_folders[i]}. Skipping.")
        continue
    
    patient_year = extract_patient_year(weight_folders[i])
    if patient_year is None:
        print(f"Warning: Could not extract patient year from {weight_folders[i]}. Skipping.")
        continue

    # try:
    #     patient_year = int(patient_year[-4:])
    # except ValueError:
    #     print(f"Warning: Invalid patient year format: {patient_year}. Skipping.")
    #     continue
    
    x.append(f"{patient_id}_{patient_year}")

In [8]:
# x
from datetime import datetime
from collections import defaultdict

# Step 1: Organize by ID
id_to_dates = defaultdict(list)
for item in x:
    id_part, date_part = item.split('_')
    date_obj = datetime.strptime(date_part, "%m-%d-%Y")
    id_to_dates[id_part].append((date_obj, item))
# print(id_to_dates)

# Step 2: Select earliest date for each ID
unique_patient = [min(dates)[1] for dates in id_to_dates.values()]

len(unique_patient)


124

In [9]:
rf_csv = r"C:\Users\Acer\Desktop\3dVS\RFcsv\volume_data7.csv"
rf_data = pd.read_csv(rf_csv, index_col=0)
# rf_data

In [10]:
import numpy as np
from skimage.transform import resize

target_spacing= [1.0, 1.0, 1.0]  # Target spacing in mm (x, y, z)

def load_spacing(image_obj):
    spacing= image_obj.GetSpacing()
    spacing= list(spacing)
    return [spacing[2], spacing[1], spacing[0]]

def resample_pair(image, label, spacing):
    shape = calculate_new_shape(spacing, image.shape)
    if check_anisotrophy(spacing):
        image = resample_anisotrophic_image(image, shape)
        if label is not None:
            label = resample_anisotrophic_label(label, shape)
    else:
        image = resample_regular_image(image, shape)
        if label is not None:
            label = resample_regular_label(label, shape)
    image = image.astype(np.float32)
    if label is not None:
        label = label.astype(np.uint8)
    return image, label

def calculate_new_shape(spacing, shape):
    spacing_ratio = np.array(spacing) / np.array(target_spacing)
    new_shape = (spacing_ratio * np.array(shape)).astype(int).tolist()
    return new_shape

def check_anisotrophy(spacing):
    def check(spacing):
        return np.max(spacing) / np.min(spacing) >= 3

    return check(spacing) or check(target_spacing)

def resample_anisotrophic_image(image, shape):
    resized = [resize_fn(i, shape[1:], 3, "edge") for i in image]
    resized = np.stack(resized, axis=0)
    resized = resize_fn(resized, shape, 0, "constant")
    return resized

def resample_anisotrophic_label(label, shape):
    depth = label.shape[0]
    reshaped = np.zeros(shape, dtype=np.uint8)
    shape_2d = shape[1:]
    reshaped_2d = np.zeros((depth, *shape_2d), dtype=np.uint8)
    n_class = np.max(label)
    for class_ in range(1, n_class + 1):
        for depth_ in range(depth):
            mask = label[depth_] == class_
            resized_2d = resize_fn(mask.astype(float), shape_2d, 1, "edge")
            reshaped_2d[depth_][resized_2d >= 0.5] = class_

    for class_ in range(1, n_class + 1):
        mask = reshaped_2d == class_
        resized = resize_fn(mask.astype(float), shape, 0, "constant")
        reshaped[resized >= 0.5] = class_
    return reshaped

def resample_regular_image(image, shape):
    resized = (resize_fn(image, shape, 3, "edge"))
    return resized

def resample_regular_label(label, shape):
    reshaped = np.zeros(shape, dtype=np.uint8)
    n_class = np.max(label)
    for class_ in range(1, n_class + 1):
        mask = label == class_
        resized = resize_fn(mask.astype(float), shape, 1, "edge")
        reshaped[resized >= 0.5] = class_
    return reshaped

def resize_fn(image, shape, order, mode):
    return resize(image, shape, order=order, mode=mode, cval=0, clip=True, anti_aliasing=False)

In [11]:
import SimpleITK as sitk
import numpy as np
reader = sitk.ImageSeriesReader()
# from utils.utils import load_spacing, resample_pair

In [12]:
def resample_segmentation_to_image(segmentation_image, image):
    resampler = sitk.ResampleImageFilter()
    resampler.SetSize(image.GetSize())
    resampler.SetOutputSpacing(image.GetSpacing())
    resampler.SetOutputDirection(image.GetDirection())
    resampler.SetOutputOrigin(image.GetOrigin())
    resampler.SetTransform(sitk.Transform())
    resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    return resampler.Execute(segmentation_image)

In [13]:
from monai.transforms import (
    Compose, NormalizeIntensityd, RandBiasFieldd,
    RandFlipd, RandRotate90d, RandGaussianNoised, RandZoomd,
    RandAdjustContrastd, RandShiftIntensityd, RandGaussianSmoothd,
    ResizeWithPadOrCropd, ToTensord
)

image_size = (128, 128, 128)

transform = Compose([
    NormalizeIntensityd(keys=["image"], nonzero=True, channel_wise=True),  # Z-normalization
    
    # Spatial augmentations
    RandZoomd(keys=["image", "mask"], min_zoom=0.9, max_zoom=1.1, prob=0.5, keep_size=True),
    RandFlipd(keys=["image", "mask"], spatial_axis=0, prob=0.3),
    RandFlipd(keys=["image", "mask"], spatial_axis=1, prob=0.3),
    RandRotate90d(keys=["image", "mask"], prob=0.3, max_k=3, spatial_axes= (0,1)),

    # Intensity augmentations
    RandGaussianNoised(keys=["image"], prob=0.5, mean=0.0, std=0.1),
    RandBiasFieldd(keys=["image"], prob=0.3, coeff_range=(0.1, 0.3), degree=3),
    RandGaussianSmoothd(keys=["image"], prob=0.5, sigma_x=(0.5, 1.5), sigma_y=(0.5, 1.5), sigma_z=(0.5, 1.5)),
    RandAdjustContrastd(keys=["image"], prob=0.5, gamma=(0.7, 1.5)),
    RandShiftIntensityd(keys=["image"], offsets=0.1, prob=0.5),  
    
    # Resize or pad/crop to final size
    ResizeWithPadOrCropd(keys=["image", "mask"], spatial_size=image_size, mode='constant'),
    ToTensord(keys=["image", "mask"]),
])

In [14]:
import matplotlib.pyplot as plt

def plot_image_and_mask(image_data, mask_data, slice_num: int = None, cmap='gray'):

    if slice_num is None:
        slice_num = image_data.shape[0] // 2

    fig, axs = plt.subplots(1, 3, figsize=(15, 5))

    axs[0].imshow(image_data[slice_num,:, :], cmap=cmap)
    axs[0].set_title(f"Image Slice {slice_num}")
    axs[0].axis('off')

    axs[1].imshow(mask_data[slice_num,:, :], cmap=cmap)
    axs[1].set_title(f"Mask Slice {slice_num}")
    axs[1].axis('off')

    axs[2].imshow(image_data[slice_num,:, :], cmap=cmap)
    axs[2].imshow(mask_data[slice_num,:, :], cmap='Reds', alpha=0.4)
    axs[2].set_title(f"Overlay Slice {slice_num}")
    axs[2].axis('off')

    plt.tight_layout()
    plt.show()

In [15]:
from typing import List, Optional, Sequence, Tuple, Union

import torch
import torch.nn as nn
from torch.nn.functional import interpolate

from monai.networks.blocks.dynunet_block import UnetBasicBlock, UnetOutBlock, UnetResBlock, UnetUpBlock

__all__ = ["DynUNet", "DynUnet", "Dynunet"]

# encoder_outputs = []
class DynUNetSkipLayer(nn.Module):

    heads: Optional[List[torch.Tensor]]

    def __init__(self, index, downsample, upsample, next_layer, heads=None, super_head=None, encoder_outputs=None):
        super().__init__()
        self.downsample = downsample
        self.next_layer = next_layer
        self.upsample = upsample
        self.super_head = super_head
        self.heads = heads
        self.index = index

        self.encoder_outputs = encoder_outputs

        # self.encoder_outputs = []

    def forward(self, x):
        downout = self.downsample(x)
        self.encoder_outputs.append(downout)
        # print(f'{self.index}- downsample output shape:- {downout.shape}')
        # print(encoder_outputs)
        nextout = self.next_layer(downout)
        upout = self.upsample(nextout, downout)
        # print(f'{self.index}- Upsample output shape:- {upout.shape}')
        if self.super_head is not None and self.heads is not None and self.index > 0:
            self.heads[self.index - 1] = self.super_head(upout)

        return upout


class DynUNet(nn.Module):

    def __init__(
        self,
        spatial_dims: int,
        in_channels: int,
        out_channels: int,
        kernel_size: Sequence[Union[Sequence[int], int]],
        strides: Sequence[Union[Sequence[int], int]],
        upsample_kernel_size: Sequence[Union[Sequence[int], int]],
        filters: Optional[Sequence[int]] = None,
        dropout: Optional[Union[Tuple, str, float]] = None,
        norm_name: Union[Tuple, str] = ("INSTANCE", {"affine": True}),
        act_name: Union[Tuple, str] = ("leakyrelu", {"inplace": True, "negative_slope": 0.01}),
        deep_supervision: bool = False,
        deep_supr_num: int = 1,
        res_block: bool = False,
        trans_bias: bool = False,
    ):
        super().__init__()
        self.spatial_dims = spatial_dims
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.strides = strides
        self.upsample_kernel_size = upsample_kernel_size
        self.norm_name = norm_name
        self.act_name = act_name
        self.dropout = dropout
        self.conv_block = UnetResBlock if res_block else UnetBasicBlock
        self.trans_bias = trans_bias
        if filters is not None:
            self.filters = filters
            self.check_filters()
        else:
            # self.filters = [min(2 ** (5 + i), 320 if spatial_dims == 3 else 512) for i in range(len(strides))]
            self.filters = [32, 64, 128, 256, 512, 1024]
        self.input_block = self.get_input_block()
        self.downsamples = self.get_downsamples()
        self.bottleneck = self.get_bottleneck()
        self.upsamples = self.get_upsamples()
        self.output_block = self.get_output_block(0)
        self.deep_supervision = deep_supervision
        self.deep_supr_num = deep_supr_num
        # initialize the typed list of supervision head outputs so that Torchscript can recognize what's going on
        self.heads: List[torch.Tensor] = [torch.rand(1)] * self.deep_supr_num
        if self.deep_supervision:
            self.deep_supervision_heads = self.get_deep_supervision_heads()
            self.check_deep_supr_num()

        self.apply(self.initialize_weights)
        self.check_kernel_stride()

        self.encoder_outputs = []

        def create_skips(index, downsamples, upsamples, bottleneck, superheads=None):

            if len(downsamples) != len(upsamples):
                raise ValueError(f"{len(downsamples)} != {len(upsamples)}")

            if len(downsamples) == 0:  # bottom of the network, pass the bottleneck block
                return bottleneck

            if superheads is None:
                next_layer = create_skips(1 + index, downsamples[1:], upsamples[1:], bottleneck)
                return DynUNetSkipLayer(index, downsample=downsamples[0], upsample=upsamples[0], next_layer=next_layer, encoder_outputs=self.encoder_outputs)

            super_head_flag = False
            if index == 0:  # don't associate a supervision head with self.input_block
                rest_heads = superheads
            else:
                if len(superheads) > 0:
                    super_head_flag = True
                    rest_heads = superheads[1:]
                else:
                    rest_heads = nn.ModuleList()

            # create the next layer down, this will stop at the bottleneck layer
            next_layer = create_skips(1 + index, downsamples[1:], upsamples[1:], bottleneck, superheads=rest_heads)
            if super_head_flag:
                return DynUNetSkipLayer(
                    index,
                    downsample=downsamples[0],
                    upsample=upsamples[0],
                    next_layer=next_layer,
                    heads=self.heads,
                    super_head=superheads[0],
                    encoder_outputs=self.encoder_outputs
                )

            return DynUNetSkipLayer(index, downsample=downsamples[0], upsample=upsamples[0], next_layer=next_layer, encoder_outputs=self.encoder_outputs)

        if not self.deep_supervision:
            self.skip_layers = create_skips(
                0, [self.input_block] + list(self.downsamples), self.upsamples[::-1], self.bottleneck
            )
        else:
            self.skip_layers = create_skips(
                0,
                [self.input_block] + list(self.downsamples),
                self.upsamples[::-1],
                self.bottleneck,
                superheads=self.deep_supervision_heads,
            )

    def check_kernel_stride(self):
        kernels, strides = self.kernel_size, self.strides
        error_msg = "length of kernel_size and strides should be the same, and no less than 3."
        if len(kernels) != len(strides) or len(kernels) < 3:
            raise ValueError(error_msg)

        for idx, k_i in enumerate(kernels):
            kernel, stride = k_i, strides[idx]
            if not isinstance(kernel, int):
                error_msg = f"length of kernel_size in block {idx} should be the same as spatial_dims."
                if len(kernel) != self.spatial_dims:
                    raise ValueError(error_msg)
            if not isinstance(stride, int):
                error_msg = f"length of stride in block {idx} should be the same as spatial_dims."
                if len(stride) != self.spatial_dims:
                    raise ValueError(error_msg)

    def check_deep_supr_num(self):
        deep_supr_num, strides = self.deep_supr_num, self.strides
        num_up_layers = len(strides) - 1
        if deep_supr_num >= num_up_layers:
            raise ValueError("deep_supr_num should be less than the number of up sample layers.")
        if deep_supr_num < 1:
            raise ValueError("deep_supr_num should be larger than 0.")

    def check_filters(self):
        filters = self.filters
        if len(filters) < len(self.strides):
            raise ValueError("length of filters should be no less than the length of strides.")
        else:
            self.filters = filters[: len(self.strides)]

    def forward(self, x):
        self.encoder_outputs.clear()
        out = self.skip_layers(x)
        out = self.output_block(out)
        if self.training and self.deep_supervision:
            out_all = [out]
            for feature_map in self.heads:
                out_all.append(interpolate(feature_map, out.shape[2:]))
            return torch.stack(out_all, dim=1)
        return self.encoder_outputs, out

    def get_input_block(self):
        return self.conv_block(
            self.spatial_dims,
            self.in_channels,
            self.filters[0],
            self.kernel_size[0],
            self.strides[0],
            self.norm_name,
            self.act_name,
            dropout=self.dropout,
        )

    def get_bottleneck(self):
        return self.conv_block(
            self.spatial_dims,
            self.filters[-2],
            self.filters[-1],
            self.kernel_size[-1],
            self.strides[-1],
            self.norm_name,
            self.act_name,
            dropout=self.dropout,
        )

    def get_output_block(self, idx: int):
        return UnetOutBlock(self.spatial_dims, self.filters[idx], self.out_channels, dropout=self.dropout)

    def get_downsamples(self):
        inp, out = self.filters[:-2], self.filters[1:-1]
        strides, kernel_size = self.strides[1:-1], self.kernel_size[1:-1]
        return self.get_module_list(inp, out, kernel_size, strides, self.conv_block)  # type: ignore

    def get_upsamples(self):
        inp, out = self.filters[1:][::-1], self.filters[:-1][::-1]
        strides, kernel_size = self.strides[1:][::-1], self.kernel_size[1:][::-1]
        upsample_kernel_size = self.upsample_kernel_size[::-1]
        return self.get_module_list(
            inp,  # type: ignore
            out,  # type: ignore
            kernel_size,
            strides,
            UnetUpBlock,  # type: ignore
            upsample_kernel_size,
            trans_bias=self.trans_bias,
        )

    def get_module_list(
        self,
        in_channels: List[int],
        out_channels: List[int],
        kernel_size: Sequence[Union[Sequence[int], int]],
        strides: Sequence[Union[Sequence[int], int]],
        conv_block: nn.Module,
        upsample_kernel_size: Optional[Sequence[Union[Sequence[int], int]]] = None,
        trans_bias: bool = False,
    ):
        layers = []
        if upsample_kernel_size is not None:
            for in_c, out_c, kernel, stride, up_kernel in zip(
                in_channels, out_channels, kernel_size, strides, upsample_kernel_size
            ):
                params = {
                    "spatial_dims": self.spatial_dims,
                    "in_channels": in_c,
                    "out_channels": out_c,
                    "kernel_size": kernel,
                    "stride": stride,
                    "norm_name": self.norm_name,
                    "act_name": self.act_name,
                    "dropout": self.dropout,
                    "upsample_kernel_size": up_kernel,
                    "trans_bias": trans_bias,
                }
                layer = conv_block(**params)
                layers.append(layer)
        else:
            for in_c, out_c, kernel, stride in zip(in_channels, out_channels, kernel_size, strides):
                params = {
                    "spatial_dims": self.spatial_dims,
                    "in_channels": in_c,
                    "out_channels": out_c,
                    "kernel_size": kernel,
                    "stride": stride,
                    "norm_name": self.norm_name,
                    "act_name": self.act_name,
                    "dropout": self.dropout,
                }
                layer = conv_block(**params)
                layers.append(layer)
        return nn.ModuleList(layers)

    def get_deep_supervision_heads(self):
        return nn.ModuleList([self.get_output_block(i + 1) for i in range(self.deep_supr_num)])

    @staticmethod
    def initialize_weights(module):
        if isinstance(module, (nn.Conv3d, nn.Conv2d, nn.ConvTranspose3d, nn.ConvTranspose2d)):
            module.weight = nn.init.kaiming_normal_(module.weight, a=0.01)
            if module.bias is not None:
                module.bias = nn.init.constant_(module.bias, 0)


DynUnet = Dynunet = DynUNet

In [16]:
model= DynUNet(spatial_dims=3,
    in_channels=1,
    out_channels=1,
    kernel_size=[3, 3, 3, 3, 3, 3],
    strides=[1, 2, 2, 2, 2, 2],
    upsample_kernel_size=[2, 2, 2, 2, 2],
    res_block=True,
).to('cpu')

In [17]:
count = 0

learnt_feature_dict = {}
for i in range(len(weight_folders)):
    patient_id = extract_patient_name(weight_folders[i])
    study_date = extract_patient_year(weight_folders[i])
    id_date = f'{patient_id}_{study_date}'

    dicom_series = reader.GetGDCMSeriesFileNames(weight_folders[i])
    reader.SetFileNames(dicom_series)
    reference_image = reader.Execute()

    segmentation_image = sitk.ReadImage(seg_paths[i])
    segmentation_resampled = resample_segmentation_to_image(segmentation_image, reference_image)

    mask_data = sitk.GetArrayFromImage(segmentation_resampled).astype(np.uint8)
    image_data = sitk.GetArrayFromImage(reference_image).astype(np.float32) 

    spacing = load_spacing(reference_image)
    image_data, mask_data = resample_pair(image_data, mask_data, spacing)

    data_dict = {
        "image": image_data[np.newaxis, ...],  # (1, D, H, W)
        "mask": mask_data[np.newaxis, ...]     # (1, D, H, W)
    }

    transformed = transform(data_dict)

    transformed_image = transformed["image"].unsqueeze(0)
    transformed_mask = transformed["mask"].unsqueeze(0)
    # print(transformed_image.shape, transformed_mask.shape)

    encoder_outputs, _ = model(transformed_image)
    learnt_feature = encoder_outputs[-1].squeeze().flatten()

    learnt_feature_dict[patient_id] = learnt_feature
    print(learnt_feature.shape)

    break


torch.Size([262144])


In [37]:
for id in unique_patient:
    id_part, date_part = id.split('_')
    learnt_feature_dict[id_part] = learnt_feature


In [38]:
learnt_feature_dict

{'VS-SEG-003': metatensor([-0.0044, -0.0047, -0.0046,  ...,  0.1734, -0.0026,  0.8034],
        grad_fn=<AliasBackward0>),
 'VS-SEG-010': metatensor([-0.0044, -0.0047, -0.0046,  ...,  0.1734, -0.0026,  0.8034],
        grad_fn=<AliasBackward0>),
 'VS-SEG-013': metatensor([-0.0044, -0.0047, -0.0046,  ...,  0.1734, -0.0026,  0.8034],
        grad_fn=<AliasBackward0>),
 'VS-SEG-021': metatensor([-0.0044, -0.0047, -0.0046,  ...,  0.1734, -0.0026,  0.8034],
        grad_fn=<AliasBackward0>),
 'VS-SEG-025': metatensor([-0.0044, -0.0047, -0.0046,  ...,  0.1734, -0.0026,  0.8034],
        grad_fn=<AliasBackward0>),
 'VS-SEG-028': metatensor([-0.0044, -0.0047, -0.0046,  ...,  0.1734, -0.0026,  0.8034],
        grad_fn=<AliasBackward0>),
 'VS-SEG-030': metatensor([-0.0044, -0.0047, -0.0046,  ...,  0.1734, -0.0026,  0.8034],
        grad_fn=<AliasBackward0>),
 'VS-SEG-031': metatensor([-0.0044, -0.0047, -0.0046,  ...,  0.1734, -0.0026,  0.8034],
        grad_fn=<AliasBackward0>),
 'VS-SEG-032': m

In [42]:
from sklearn.decomposition import PCA

deep_features = torch.stack(list(learnt_feature_dict.values())).detach().cpu().numpy() 

pca = PCA(n_components=50, random_state=42)
reduced_features = pca.fit_transform(deep_features)

In [41]:
reduced_features.shape

(124, 50)

In [5]:
import pandas as pd
dest_path = r'C:\Users\Acer\Desktop\3dVS\RFcsv\volume_data5.csv'
volume_data = pd.read_csv(dest_path, index_col=0)
volume_data

,volume_diff,change,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
Patient_ID,,,,,,,,,,,,,,,,,,,,,
VS-MC-RC-001,-51.0,0,0.574892,0.420573,4.667643,11.098302,7.692140,10.667506,9.907894,11.824483,...,10.695441,0.076159,4.020056,0.045500,8.521760e+03,14.322749,0.005272,16.255589,0.019169,0.263056
VS-MC-RC-002,1069.0,1,0.574886,0.383779,4.624665,12.050322,14.526802,8.669793,12.282286,14.842891,...,480.178708,0.004672,6.442091,0.612416,1.898097e+00,0.149696,0.005885,6488.583068,0.304126,7.944378
VS-MC-RC-004,-13391.0,0,0.767449,0.750977,24.363244,32.442076,37.514031,33.636311,35.544042,38.487053,...,32.093094,0.029082,7.190271,0.010443,3.126897e+06,61.614317,0.000073,62.767139,0.027590,0.011189
VS-MC-RC-005,2256.0,1,0.785051,0.676358,10.682136,15.793601,19.495446,15.119270,20.937774,21.244994,...,1403.268164,0.000975,7.284012,0.535631,4.696216e+00,0.178940,0.000656,57029.297654,0.081517,12.098159
VS-MC-RC-006,205.0,1,0.560217,0.390187,8.214488,21.052694,23.263138,14.840822,24.317952,24.568533,...,35.924964,0.077624,5.696608,0.077204,1.368253e+04,7.544970,0.000963,164.020212,0.028489,0.178225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VS-SEG-195,NaN,0,0.473118,0.406433,4.480535,11.024049,11.574484,6.088697,10.146551,11.833204,...,2591.392859,0.001952,6.126663,0.862069,2.944000e-01,0.013151,0.023994,27866.168951,2.382383,50.594926
VS-SEG-197,1251.0,1,0.763451,0.612028,9.262721,15.134468,17.845172,13.550974,17.346565,19.078826,...,112.478485,0.010389,6.023579,0.258682,3.014484e+02,0.474109,0.004280,536.160015,0.123540,0.938516
VS-SEG-199,1102.0,1,0.880602,0.623155,12.667649,20.328248,25.398081,20.264196,27.537262,30.923181,...,17.464088,0.078385,5.835668,0.034917,2.251998e+05,88.305997,0.000146,179.378541,0.017395,0.053233


In [6]:
selected_feature_names_rfe = ['volume_diff', 'change', 'original_shape_MajorAxisLength', 'original_shape_Maximum2DDiameterRow', 'original_shape_SurfaceVolumeRatio', 'original_glcm_Correlation', 'original_glcm_MCC', 'original_gldm_DependenceNonUniformity', 'original_gldm_GrayLevelNonUniformity', 'original_glszm_LargeAreaHighGrayLevelEmphasis', 'original_ngtdm_Coarseness','original_ngtdm_Strength']
filtered_df = volume_data[selected_feature_names_rfe]
filtered_df

,volume_diff,change,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterRow,original_shape_SurfaceVolumeRatio,original_glcm_Correlation,original_glcm_MCC,original_gldm_DependenceNonUniformity,original_gldm_GrayLevelNonUniformity,original_glszm_LargeAreaHighGrayLevelEmphasis,original_ngtdm_Coarseness,original_ngtdm_Strength
Patient_ID,,,,,,,,,,,,
VS-MC-RC-001,-51.0,0,11.098302,10.667506,1.258267,0.224535,0.324320,53.124629,419.354105,1.741270e+04,0.005272,0.263056
VS-MC-RC-002,1069.0,1,12.050322,8.669793,1.197842,0.266649,0.454266,380.436242,43.684564,2.363640e+03,0.005885,7.944378
VS-MC-RC-004,-13391.0,0,32.442076,33.636311,0.350630,0.937206,0.941602,16772.235332,38692.902773,3.764821e+08,0.000073,0.011189
VS-MC-RC-005,2256.0,1,15.793601,15.119270,0.622690,0.532240,0.817496,3718.285922,422.397406,1.321722e+04,0.000656,12.098159
VS-MC-RC-006,205.0,1,21.052694,14.840822,0.707721,0.547405,0.581540,815.913548,1288.864086,5.086712e+05,0.000963,0.178225
...,...,...,...,...,...,...,...,...,...,...,...,...
VS-SEG-195,NaN,0,11.024049,6.088697,1.132370,0.114844,0.974734,108.379310,3.367816,5.449440e+03,0.023994,50.594926
VS-SEG-197,1251.0,1,15.134468,13.550974,0.589559,0.451247,0.490837,296.263580,164.133571,1.087350e+05,0.004280,0.938516
VS-SEG-199,1102.0,1,20.328248,20.264196,0.537973,0.485690,0.519013,4153.128225,9272.754838,2.321828e+06,0.000146,0.053233


In [7]:
dest_path = r'C:\Users\Acer\Desktop\3dVS\RFcsv\volume_data8.csv'
filtered_df.to_csv(dest_path)

In [49]:
import pandas as pd
df = pd.DataFrame(reduced_features, index=list(learnt_feature_dict.keys()))

In [57]:
merged_df = pd.concat([volume_data, df], axis=1)

In [58]:
merged_df

,volume_diff,change,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,...,40,41,42,43,44,45,46,47,48,49
VS-MC-RC-001,-51.0,0,0.574892,0.420573,4.667643,11.098302,7.692140,10.667506,9.907894,11.824483,...,5.092367e-11,-3.311039e-11,2.061937e-11,4.778542e-11,7.418246e-11,2.500083e-11,-8.782585e-11,-6.378301e-11,-4.610849e-11,4.201538e-11
VS-MC-RC-002,1069.0,1,0.574886,0.383779,4.624665,12.050322,14.526802,8.669793,12.282286,14.842891,...,-1.608281e-11,-1.847567e-11,8.310458e-12,1.186691e-11,-1.325892e-11,9.744678e-12,-1.202614e-11,1.438493e-12,-9.018990e-12,-1.540164e-11
VS-MC-RC-004,-13391.0,0,0.767449,0.750977,24.363244,32.442076,37.514031,33.636311,35.544042,38.487053,...,-7.182792e-11,-7.999611e-11,1.374990e-11,-2.494009e-11,2.011064e-12,1.380076e-11,2.545963e-11,-3.038174e-11,2.631792e-11,4.007245e-13
VS-MC-RC-005,2256.0,1,0.785051,0.676358,10.682136,15.793601,19.495446,15.119270,20.937774,21.244994,...,5.158537e-11,-3.143070e-11,1.995326e-11,4.720065e-11,7.348847e-11,2.422828e-11,-8.701004e-11,-6.240241e-11,-4.562404e-11,4.158858e-11
VS-MC-RC-006,205.0,1,0.560217,0.390187,8.214488,21.052694,23.263138,14.840822,24.317952,24.568533,...,-7.142954e-11,-7.850021e-11,1.321436e-11,-2.533388e-11,1.063248e-12,1.318553e-11,2.619215e-11,-2.899279e-11,2.665566e-11,-2.287335e-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VS-SEG-195,NaN,0,0.473118,0.406433,4.480535,11.024049,11.574484,6.088697,10.146551,11.833204,...,-4.980029e-11,6.753755e-11,7.596522e-11,-4.069876e-12,-3.514847e-11,3.668045e-11,-1.604955e-11,1.305974e-10,-3.959222e-11,6.960967e-11
VS-SEG-197,1251.0,1,0.763451,0.612028,9.262721,15.134468,17.845172,13.550974,17.346565,19.078826,...,-1.138869e-12,4.548945e-12,-1.264045e-11,-4.607245e-11,-1.189248e-11,4.289608e-11,-2.557767e-11,-1.233988e-11,2.286994e-11,2.623036e-11
VS-SEG-199,1102.0,1,0.880602,0.623155,12.667649,20.328248,25.398081,20.264196,27.537262,30.923181,...,-4.908526e-11,6.655519e-11,7.513616e-11,-3.493213e-12,-3.445729e-11,3.563636e-11,-4.650824e-12,5.993710e-11,-6.002503e-11,-1.121451e-10
VS-SEG-218,1870.0,1,0.942108,0.737152,17.979514,24.390526,27.617950,27.083334,30.665712,31.787093,...,8.600694e-12,3.286795e-12,-3.388779e-11,-9.084699e-12,-2.804953e-11,-2.319156e-11,1.821559e-11,6.168832e-12,2.456254e-11,-1.258325e-11
